In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

# Define paths to your dataset
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Define image size and batch size
img_size = (299, 299)
batch_size = 32

# Create data generators with augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Create data generator without augmentation for validation set
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load InceptionV3 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Build a custom model on top of the InceptionV3 base
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save('nude_safe_classifier.h5')

Found 19999 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Epoch 1/10
 39/624 [>.............................] - ETA: 3:29 - loss: 0.4198 - accuracy: 0.8077

/usr/lib/python3/dist-packages/PIL/Image.py:2896: DecompressionBombWarning: Image size (138012160 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


624/624 [==============================] - 268s 426ms/step - loss: 0.2704 - accuracy: 0.8915 - val_loss: 0.2040 - val_accuracy: 0.9195
Epoch 2/10
624/624 [==============================] - 254s 407ms/step - loss: 0.2205 - accuracy: 0.9140 - val_loss: 0.1901 - val_accuracy: 0.9265
Epoch 3/10
624/624 [==============================] - 252s 404ms/step - loss: 0.2137 - accuracy: 0.9166 - val_loss: 0.1873 - val_accuracy: 0.9287
Epoch 4/10
624/624 [==============================] - 253s 405ms/step - loss: 0.2062 - accuracy: 0.9206 - val_loss: 0.1867 - val_accuracy: 0.9300
Epoch 5/10
624/624 [==============================] - 252s 404ms/step - loss: 0.1967 - accuracy: 0.9219 - val_loss: 0.2019 - val_accuracy: 0.9185
Epoch 6/10
624/624 [==============================] - 254s 407ms/step - loss: 0.1952 - accuracy: 0.9216 - val_loss: 0.1792 - val_accuracy: 0.9315
Epoch 7/10
624/624 [==============================] - 253s 405ms/step - loss: 0.1890 - accuracy: 0.9278 - val_loss: 0.1774 - val_accura

/home/tinkerspace/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!ls

dataset  dataset.zip  model.ipynb  new-model.ipynb  nude_safe_classifier.h5


In [ ]:
!unzip '/content/drive/MyDrive/mini project/reduced_dataset.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/mini project/nude_safe_classifier.h5')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_dir = 'dataset/test'
img_size = (299, 299)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=1,  # Set batch_size to 1 to process images one by one
    class_mode='binary',
    shuffle=False  # Ensure images are processed in order
)


Found 4000 images belonging to 2 classes.


In [ ]:
# Evaluate the model on the test set
results = model.evaluate(test_generator)

# Print the evaluation results
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])


4000/4000 [==============================] - 73s 16ms/step - loss: 0.1744 - accuracy: 0.9345
Test Loss: 0.17443981766700745
Test Accuracy: 0.934499979019165


In [ ]:
import numpy as np

# Reset the test generator to start from the beginning
test_generator.reset()

# Get predictions for each image in the test set
predictions = model.predict(test_generator)

# Convert predictions to binary labels (0 or 1)
binary_predictions = np.round(predictions)

# Print the predictions for each image
for i in range(len(test_generator.filenames)):
    filename = test_generator.filenames[i]
    true_label = test_generator.classes[i]
    predicted_label = int(binary_predictions[i])

    print(f"Filename: {filename}, True Label: {true_label}, Predicted Label: {predicted_label}")


4000/4000 [==============================] - 54s 14ms/step
Filename: nude/prefix_GantMan_0006B35A-6AC5-4EB0-87F4-16D2F2FE37CC.jpg.jpeg, True Label: 0, Predicted Label: 0
Filename: nude/prefix_GantMan_003DB635-B080-46A9-A457-2EF7D4EF213B.jpg.jpeg, True Label: 0, Predicted Label: 1
Filename: nude/prefix_GantMan_009266FC-4B45-4396-9207-B9D1991ED3E6.jpg.jpeg, True Label: 0, Predicted Label: 0
Filename: nude/prefix_GantMan_00AB0587-5110-44C1-9A1D-229487595140.jpg.jpeg, True Label: 0, Predicted Label: 0
Filename: nude/prefix_GantMan_00E8D797-C189-49F8-9D92-4B2EED4E59E2.jpg.jpeg, True Label: 0, Predicted Label: 0
Filename: nude/prefix_GantMan_013A2343-A459-4B2A-8FBE-382D41E2EEE0.jpg.jpeg, True Label: 0, Predicted Label: 0
Filename: nude/prefix_GantMan_02189FA7-D82D-4DF7-B664-9DB7D066127E.jpg.jpeg, True Label: 0, Predicted Label: 1
Filename: nude/prefix_GantMan_046FF6B3-2EB8-4BF7-A360-07C30A57A9B0.jpg.jpeg, True Label: 0, Predicted Label: 1
Filename: nude/prefix_GantMan_04B6E037-48E5-4310-8CA0

In [ ]:
import numpy as np

# Reset the test generator to start from the beginning
test_generator.reset()

# Get predictions for each image in the test set
predictions = model.predict(test_generator)

# Convert predictions to binary labels (0 or 1)
binary_predictions = np.round(predictions)

# Get true labels from the test generator
true_labels = test_generator.classes

# Calculate accuracy
accuracy = np.mean(binary_predictions == true_labels)

# Print the overall accuracy
print("Test Accuracy:", accuracy)


4000/4000 [==============================] - 54s 14ms/step
Test Accuracy: 0.5
